# Evaluate DR

In [6]:
# load the dataset
import os
from beir.datasets.data_loader import GenericDataLoader
from utils import data_path

dataset = 'scifact'
data_path = os.path.join(data_path, dataset)

corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

print(len(queries), 'questions')
print(len(corpus), 'documents')

300 questions
5183 documents


In [7]:
# load pre-trained model
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval import models

model_name = 'msmarco-distilbert-base-tas-b'
model = DRES(models.SentenceBERT(model_name), batch_size=16)
retriever = EvaluateRetrieval(model, score_function="dot")
results = retriever.retrieve(corpus, queries)

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

Batches:   0%|          | 0/324 [00:00<?, ?it/s]

In [13]:
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
print('%.3f\t'*6 %(ndcg['NDCG@10'], recall['Recall@1'], recall['Recall@5'],
                   recall['Recall@10'],recall['Recall@100'], recall['Recall@1000']))

0.643	0.509	0.700	0.761	0.891	0.983	


# Evaluate BM25

In [14]:
# use pyserini to retrieve relevant documents with default BM25 
from pyserini.search import SimpleSearcher
from beir.retrieval.evaluation import EvaluateRetrieval

from utils import indices_path

index_name = "beir/scifact"
path_index = indices_path + index_name

retriever = EvaluateRetrieval()
qids = list(queries)
query_texts = [queries[qid] for qid in qids]
payload = {"queries": query_texts, "qids": qids, "k": max(retriever.k_values)}

searcher = SimpleSearcher(path_index)
hits = searcher.batch_search(queries=query_texts, qids=qids, k=max(retriever.k_values))

results = {}
for id_ in qids:
    results[id_] = {}
    for hit in hits[id_]:
        results[id_][hit.docid.strip()] = hit.score

In [15]:
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
print('%.3f\t'*6 %(ndcg['NDCG@10'], recall['Recall@1'], recall['Recall@5'],
                   recall['Recall@10'],recall['Recall@100'], recall['Recall@1000']))

0.663	0.514	0.731	0.791	0.917	0.973	


In [5]:
# show examples
import random

query_id, scores_dict = random.choice(list(results.items()))

scores = sorted(scores_dict.items(), key=lambda item: item[1], reverse=True)
for rank in range(10):
    doc_id = scores[rank][0]
    print("Doc %d: %s [%s] - %s\n" % (rank+1, doc_id, corpus[doc_id].get("title"), corpus[doc_id].get("text")))

Doc 1: 2988714 [Coupled local translation and degradation regulate growth cone collapse] - Local translation mediates axonal responses to Semaphorin3A (Sema3A) and other guidance cues. However, only a subset of the axonal proteome is locally synthesized, whereas most proteins are trafficked from the soma. The reason why only specific proteins are locally synthesized is unknown. Here we show that local protein synthesis and degradation are linked events in growth cones. We find that growth cones exhibit high levels of ubiquitination and that local signalling pathways trigger the ubiquitination and degradation of RhoA, a mediator of Sema3A-induced growth cone collapse. Inhibition of RhoA degradation is sufficient to remove the protein-synthesis requirement for Sema3A-induced growth cone collapse. In addition to RhoA, we find that locally translated proteins are the main targets of the ubiquitin-proteasome system in growth cones. Thus, local protein degradation is a major feature of growt